## Idea testing notebook for CA4023 Assignment 1

Here, I test out ideas for in the assignment.

The full movie reviews which I analysed for Part 3 of the assignment can be found in this notebook, in Section 3.2

### 1. Sentence Boundary Detection



In [78]:
import re
import io

In [79]:
f = io.open("part-one/example1.txt", mode="r", encoding="utf-8")
example = f.read().replace('\n', ' ')
f.close()

In [80]:
def add_newline(match_obj):
    return match_obj.group(0)+'\n'

In [81]:
example

"The difference for them can be significant. A 10-year loan of €50,000 from you at zero interest would involve monthly payments of €417, or €5,000 a year. With An Post’s market-leading rate, the friend would be paying €525.45 a month, or over €6,300 a year and over €13,000 interest over the life of the loan.  To test the effectiveness of this sentence boundary detector, let's try some special cases!!! Ironic huh? That was a special case... So is this, and this too! Pi being 3.14159 is not really an edge case. Two more things: The capital of U.s.a. is Washington DC, but that isn't the end of a sentence! And George W. Bush was the 43rd president of America, but that wasn't a sentence ending either. How will we deal with those pesky edge cases??? Disclaimer: All characters are ficitonal. I went to Dr. Larry because I had a sore head, and he said Prof. Phillip's and Mrs. Mcgeoghs books had an antidote for that. I.e. liquid ibuprofen. J. K. Rowling wrote Harry Potter, but those initials did

In [86]:
# naiive sentence boundary detection
abbrv2 = 'Mr|Ms|Dr'
abbrv3 = 'Mrs|[Ee]\.[Gg]|[Ii]\.[Ee]|[Ee]tc'
abbrv4 = 'Prof|Miss'
initial='\s[A-Z]'

# check that previous word is not an abbreviation
lookbehind_abbrv='(?<!'+abbrv2+')(?<!'+abbrv3+')(?<!'+abbrv4+')' 

# check that previous word is not a single capital letter
lookbehind_initial='(?<!'+initial+')'

# check that sentence end is not followed by a lowercase letter
lookahead='(?![a-z])'

sentence_end_punct='[?!:]+\s+'

# this pattern is separate as initials are only followed by a period (J. K. Rowling, not J: K: Rowling)
sentence_end_period='[\.]+\s+'

end_re1 = lookbehind_abbrv + sentence_end_punct + lookahead  # main pattern match
end_re2 = lookbehind_initial + lookbehind_abbrv + sentence_end_period + lookahead # for catching initials
print(re.sub(end_re1+"|"+end_re2, add_newline, example))

The difference for them can be significant. 
A 10-year loan of €50,000 from you at zero interest would involve monthly payments of €417, or €5,000 a year. 
With An Post’s market-leading rate, the friend would be paying €525.45 a month, or over €6,300 a year and over €13,000 interest over the life of the loan.  
To test the effectiveness of this sentence boundary detector, let's try some special cases!!! 
Ironic huh? 
That was a special case... 
So is this, and this too! 
Pi being 3.14159 is not really an edge case. 
Two more things: 
The capital of U.s.a. is Washington DC, but that isn't the end of a sentence! 
And George W. Bush was the 43rd president of America, but that wasn't a sentence ending either. 
How will we deal with those pesky edge cases??? 
Disclaimer: 
All characters are ficitonal. 
I went to Dr. Larry because I had a sore head, and he said Prof. Phillip's and Mrs. Mcgeoghs books had an antidote for that. 
I.e. liquid ibuprofen. 
J. K. Rowling wrote Harry Potter, but tho

- Sentences can end with different forms of punctuation "[.?!:]" .
- Mr., Mrs., Ms., Miss., Dr., Prof., are not sentence endings, but rather honorifics.

To capture as many normal cases as possible, I will assume the following:
- Correct sentence endings are always used in the input text files. That is, a space must follow a sentence-end punctuation to be considered a sentence ending, and sentences do not start with lowercase letters.
-  A sentence does not end with a single capital letter. As a result of this assumption, the rare case in which it does end with a capital letter will not be detected.


In [85]:
def sentence_split(text):
    '''
        Given text as input, return each sentence on a separate line.

    '''
    # known abbreviations of different lengths
    abbrv2 = 'Mr|Ms|Dr'
    abbrv3 = 'Mrs|[Ee]\.[Gg]|[Ii]\.[Ee]|[Ee]tc'
    abbrv4 = 'Prof|Miss'

    initial='\s[A-Z]'

    # check that previous word is not an abbreviation
    lookbehind_abbrv='(?<!'+abbrv2+')(?<!'+abbrv3+')(?<!'+abbrv4+')' 

    # check that previous word is not a single capital letter (assumed to be an initial)
    lookbehind_initial='(?<!'+initial+')'

    # check that sentence end is not followed by a lowercase letter
    lookahead='(?![a-z])'

    # matches sentences ending in non-period punctuation
    sentence_end_punct='[?!:]+\s+'

    # separate sentence end pattern as initials are only followed by a period (J. K. Rowling, not J: K: Rowling)
    sentence_end_period='[\.]+\s+'

    end_re1 = lookbehind_abbrv + sentence_end_punct + lookahead  # punctuation pattern match
    end_re2 = lookbehind_initial + end_re1 # for catching initials

    return print(re.sub(end_re1+"|"+end_re2, add_newline, example))
    
sentence_split(example)

The difference for them can be significant. A 10-year loan of €50,000 from you at zero interest would involve monthly payments of €417, or €5,000 a year. With An Post’s market-leading rate, the friend would be paying €525.45 a month, or over €6,300 a year and over €13,000 interest over the life of the loan.  To test the effectiveness of this sentence boundary detector, let's try some special cases!!! 
Ironic huh? 
That was a special case... So is this, and this too! 
Pi being 3.14159 is not really an edge case. Two more things: 
The capital of U.s.a. is Washington DC, but that isn't the end of a sentence! 
And George W. Bush was the 43rd president of America, but that wasn't a sentence ending either. How will we deal with those pesky edge cases??? 
Disclaimer: 
All characters are ficitonal. I went to Dr. Larry because I had a sore head, and he said Prof. Phillip's and Mrs. Mcgeoghs books had an antidote for that. I.e. liquid ibuprofen. J. K. Rowling wrote Harry Potter, but those initia

### 2. Bigram Language Model

In [1]:
from collections import defaultdict
corpus = ['<s> b c </s>', '<s> c c </s>', '<s> c b </s>', '<s> b b </s>']

In [6]:
# initialise dictionaries for storing frequency. These are used for computing probabilities

def train(corpus):
    '''
    Given a training corpus in the form of a list of sentences, return word and bigram frequency dictionaries
    for computing probabilities
    '''     
    word_freq = defaultdict(int)
    bigram_freq = defaultdict(int)

    for sentence in corpus:
        tokens = sentence.split()
        
        # update word frequency dicitonary
        for token in tokens:
            word_freq[token] += 1
        
        # update bigram frequency dictionary
        if len(tokens) >= 2:
            i = 0
            while i < len(tokens)-1:
                bigram_freq[tokens[i]+' '+tokens[i+1]] += 1
                i += 1
    
    print("Returning 2 dicitonaries. word_freq [0] and bigram_freq [1].")
    return word_freq, bigram_freq

word_freq, bigram_freq = train(corpus)

Returning 2 dicitonaries. word_freq [0] and bigram_freq [1].


In [7]:
print(word_freq)
print(bigram_freq)

defaultdict(<class 'int'>, {'<s>': 4, 'b': 4, 'c': 4, '</s>': 4})
defaultdict(<class 'int'>, {'<s> b': 2, 'b c': 1, 'c </s>': 2, '<s> c': 2, 'c c': 1, 'c b': 1, 'b </s>': 2, 'b b': 1})


In [9]:
# computing probabilities
def compute_prob(sentence, word_freq=word_freq, bigram_freq=bigram_freq):
    ''' 
    Given bigram and word frequencies from a coprus, return the probability of the input sentence.

    In a bigram model, P(w1, w2, ..., wn) = P(w1)*P(w2|w1)*P(w3|w2)*...*P(wn|wn-1).
    Since the model uses word frequencies in the corpus to approximate probabilities:
    P(wn|wn-1) = (Count of [wn-1, wn] bigram in corpus) / Count of wn-1 in corpus.

    '''
    n_corpus_tokens = sum(word_freq.values())
    tokens = sentence.split()
    i = 0
    prob = 1
    while i < len(tokens):
        curr = tokens[i]
        if i == 0:  # edge case: start of sentence
            prob *= (word_freq[curr]) / n_corpus_tokens
        else:
            prev = tokens[i-1]
            prob *= (bigram_freq[prev+" "+curr])/(word_freq[prev])
        i += 1
    
    return prob
compute_prob("<s> c </s>")

0.0625

### 3. Naiive Bayes Sentiment Polarity Classifier

#### 3.1 Model build

In [1]:
# create dataset from the text files
import pandas as pd
import numpy as np
import os
import sys
from collections import defaultdict
import string
import math
seed=637

pos_path = '../data/txt_sentoken/pos'
neg_path = '../data/txt_sentoken/neg'
pos_df = pd.DataFrame()
neg_df = pd.DataFrame()

# parse the files in positive directory
for filename in os.listdir(pos_path):
    with open(os.path.join(pos_path, filename)) as f:
        review = f.read()
        label='positive'
        
        # assign to train and test set depending on the filename
        which_set = 'train' if int(filename[2]) < 9 else 'test'
        row = pd.DataFrame({'review':[review], 'label':[label], 'set':[which_set]})

    pos_df = pd.concat([pos_df, row], axis=0)

# parse the files in negative directory
for filename in os.listdir(neg_path):
    with open(os.path.join(neg_path, filename)) as f:
        review = f.read()
        label='negative'
        
        # assign to train and test set depending on the filename
        which_set = 'train' if int(filename[2]) < 9 else 'test'
        row = pd.DataFrame({'review':[review], 'label':[label], 'set':[which_set]})

    neg_df = pd.concat([neg_df, row], axis=0)

df = pd.concat([pos_df, neg_df], axis=0).reset_index(drop=True)
train_df = df.loc[df.set=='train', :].drop(columns=['set'])
test_df = df.loc[df.set=='test', :].drop(columns=['set'])

In [2]:
train_df.head()

,review,label
0,films adapted from comic books have had plenty...,positive
1,every now and then a movie comes along from a ...,positive
2,you've got mail works alot better than it dese...,positive
3,""" jaws "" is a rare film that grabs your atten...",positive
4,moviemaking is a lot like being the general ma...,positive


In [3]:

def run_nb_model(train_df=train_df, test_df=test_df, precision=8):
    '''
        Given a training and testing dataframe as input containing columns [review, label], return the label predictions of
        the test data. If the testing dataframe has a label column, use it to compute the accuracy of the classifier. Otherwise, 
        just return the predictions.

        Arguments:
            train_df (pd.DataFrame): data frame with training reviews and their "positive" or "negative" labels
            test_df (pd.DataFrame): data frame with testing reviews. The predicted labels are appended to test_df when this function is run.
            precision (int): decimal point precision with which to return float values.
        
        Returns:
            predicted labels for test_df, and optionally the accuracy as a float with the specified precision.
    '''
    def tokenize_review(review):
        ''' Helper function to return tokens from a review as a list. '''
        tokens = []
        for token in review.split():
            token = token.strip(string.punctuation+' ').lower()
            
            # ignore empty tokens
            if token != ' ' and token != '':
                tokens.append(token)
        
        return tokens
    
    def calc_accuracy(y_true, y_pred):
        ''' Helper function to compute model accuracy given true labels and predictions '''
        n_correct = 0
        i = 0
        while i < len(y_true):
            if y_true[i] == y_pred[i]:
                n_correct += 1
            i += 1
        return n_correct / len(y_true)

    # build word count dictionaries for positive and negative reviews.
    pos_freq = defaultdict(int)
    neg_freq = defaultdict(int)

    # parse through each review in training data, tokenize and add tokens to their respective dictionaries
    for index, row in train_df.iterrows():
        for token in tokenize_review(row.review):
            # add tokens to the word counts
            if row.label == 'positive':
                pos_freq[token] += 1
            else:
                neg_freq[token] += 1
    
    n_pos = sum(pos_freq.values())
    n_neg = sum(neg_freq.values())
    
    # find probability of review from both positive and negative frequency distributions.
    predictions = []
    lp_pos = []
    lp_neg = []
    for index, row in test_df.iterrows():
        log_prob_pos = 0
        log_prob_neg = 0
        for token in tokenize_review(row.review):
            # ignores tokens that didn't appear in the training reviews
            if pos_freq[token] > 0:
                log_prob_pos += math.log(pos_freq[token]/n_pos)
            if neg_freq[token] > 0:
                log_prob_neg += math.log(neg_freq[token]/n_neg)
        
        # whichever probability is highest determines the predicted label.
        ans = 'positive' if log_prob_pos >= log_prob_neg else 'negative'
        predictions.append(ans)
        lp_pos.append(log_prob_pos)
        lp_neg.append(log_prob_neg)

    
    # add the predictions, and log probabilities to the test dataframe for analysis
    test_df['pred_label'] = predictions
    test_df['logp_positive'] = lp_pos
    test_df['logp_negative'] = lp_neg
    if 'label' in test_df.columns:
        accuracy = round(calc_accuracy(y_true=test_df.label.tolist(), y_pred=predictions), precision)

    # returning pos_freq and neg_freq for checking sentiment of individual words in the next section
    return predictions, accuracy, pos_freq, neg_freq

    
predictions, accuracy, pos_freq, neg_freq = run_nb_model(train_df, test_df, precision=8)

In [71]:
accuracy

0.505

#### 3.2 Analysing predictions from Naive Bayes model

In [16]:
correct_sample = test_df.loc[test_df.label == test_df.pred_label].sample(5, random_state=seed)
incorrect_sample = test_df.loc[test_df.label != test_df.pred_label].sample(5, random_state=seed)

In [20]:
correct_sample['log_p_diff'] = np.abs(correct_sample.logp_positive - correct_sample.logp_negative)
incorrect_sample['log_p_diff'] = np.abs(incorrect_sample.logp_positive - incorrect_sample.logp_negative)

In [35]:
correct_sample.round()

,review,label,pred_label,logp_positive,logp_negative,log_p_diff
1909,walken stars as a mobster who is kidnapped and...,negative,negative,-2500.0,-2480.0,20.0
1964,ever feel you're spending your whole life on t...,negative,negative,-3703.0,-3664.0,39.0
1929,arye cross and courteney cox star as a pair of...,negative,negative,-1888.0,-1827.0,61.0
1974,"long ago , films were constructed of strong di...",negative,negative,-2847.0,-2822.0,25.0
1998,there was a time when john carpenter was a gre...,negative,negative,-3658.0,-3597.0,61.0


In [27]:
incorrect_sample.round()

,review,label,pred_label,logp_positive,logp_negative,log_p_diff
987,trees lounge is the directoral debut from one ...,positive,negative,-1344.0,-1330.0,14.0
1961,"barb wire , pamela anderson lee's first foray ...",negative,positive,-2235.0,-2272.0,37.0
1914,woof ! too bad that leap of faith was the titl...,negative,positive,-2947.0,-2951.0,4.0
1969,capsule : john the baptist is sent from heaven...,negative,positive,-1604.0,-1612.0,8.0
1977,i saw this film on christmas day expecting an ...,negative,positive,-3124.0,-3141.0,17.0


In [91]:
# Full reviews which are analysed in ANALYSIS.md
# output may be too long to view here, should be optional to view in a text editor
print("CORRECT PREDICTIONS\n"+"-"*100)
for index, row in correct_sample.iterrows():
    print("label:", row.label, "\npredicted:", row.pred_label)
    print("Review:\n"+row.review+"\n")
    print("-"*100+"\n")

print("\nINCORRECT PREDICTIONS\n"+"-"*100)
# Full reviews which are analysed in ANALYSIS.md
# output may be too long to view here, should be optional to view in a text editor
for index, row in incorrect_sample.iterrows():
    print("label:", row.label, "\npredicted:", row.pred_label)
    print("Review:\n"+row.review+"\n")
    print("-"*100+"\n")


CORRECT PREDICTIONS
----------------------------------------------------------------------------------------------------
label: negative 
predicted: negative
Review:
walken stars as a mobster who is kidnapped and held for ransom by four bratty rich kids . 
it seems that a woman has also been kidnapped--she is the sister of one of them ( e . t . 's 
henry thomas ) and the girlfriend of another ( flannery ) --and the asking price is $2 million , which said snots are unable to cough up alone . 
they even cut off walken's finger to show they mean business , because they are desperate to save the woman's life . 
suicide kings is a terrible film . 
walken aside , there isn't a single appealing cast member . 
o'fallon creates characters that are functional types without any resonance . 
in an amusingly unironic scene , walken plays poker with the foursome and describes each of their personalities to a tee--it's as if he was reading the summary sheet for a casting director . 
the plot is anoth

In [13]:
n_pos_tokens = sum(pos_freq.values())
n_neg_tokens = sum(neg_freq.values())
print("n_tokens in positive reviews:", n_pos_tokens)
print("n_tokens in negative reviews:", n_neg_tokens)


n_tokens in positive reviews: 613316
n_tokens in negative reviews: 547676


In [5]:
def check_prob(word, pos_freq=pos_freq, neg_freq=neg_freq, precision=10):
    ''' Function to check the probability that a word occurs in a positive or negative review.
        Uses word frequency dictionaries to compute the probabilities.

        Returns:
        probability of word occuring given review is positive[0]/negative[1]
    '''
    n_pos_tokens = sum(pos_freq.values())
    n_neg_tokens = sum(neg_freq.values())
    prob_pos = round(pos_freq[word]/n_pos_tokens, precision)
    prob_neg = round(neg_freq[word]/n_neg_tokens, precision)

    return prob_pos, prob_neg

In [6]:
# Example 1 candidate words
candidates = ['desparate', 'terrible', 'issue', 'bizarre']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))    

desparate
Probability in positive review: 0.0
Probability in negarive review: 1.8259e-06

terrible
Probability in positive review: 4.4023e-05
Probability in negarive review: 0.0001789379

issue
Probability in positive review: 3.91315e-05
Probability in negarive review: 4.01697e-05

bizarre
Probability in positive review: 9.13069e-05
Probability in negarive review: 8.9469e-05



In [7]:
# Example 2 candidate words
candidates = ['settles', 'dysfunctional', 'too', 'satire']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))    
    

settles
Probability in positive review: 9.7829e-06
Probability in negarive review: 1.09554e-05

dysfunctional
Probability in positive review: 1.46743e-05
Probability in negarive review: 7.3036e-06

too
Probability in positive review: 0.0010793783
Probability in negarive review: 0.0013402084

satire
Probability in positive review: 8.1524e-05
Probability in negarive review: 5.84287e-05



In [8]:
# Example 3 candidate words
candidates = ['goofy', 'waste', 'stereotypical', 'bad']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))    
    

goofy
Probability in positive review: 5.21754e-05
Probability in negarive review: 6.39064e-05

waste
Probability in positive review: 3.58706e-05
Probability in negarive review: 0.0001935451

stereotypical
Probability in positive review: 2.11963e-05
Probability in negarive review: 4.19956e-05

bad
Probability in positive review: 0.0005299063
Probability in negarive review: 0.001685303



In [9]:
# Example 4 candidate words
candidates = ['terrible', 'awful', 'pathetic', 'ugly']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))   

terrible
Probability in positive review: 4.4023e-05
Probability in negarive review: 0.0001789379

awful
Probability in positive review: 3.26096e-05
Probability in negarive review: 0.0001734602

pathetic
Probability in positive review: 4.56535e-05
Probability in negarive review: 0.0001077279

ugly
Probability in positive review: 3.91315e-05
Probability in negarive review: 7.121e-05



In [10]:
# Example 5 candidate words
candidates = ['lost', 'unscary', 'stylelessly', 'disheartening']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))   

lost
Probability in positive review: 0.0002673989
Probability in negarive review: 0.0003688312

unscary
Probability in positive review: 1.6305e-06
Probability in negarive review: 3.6518e-06

stylelessly
Probability in positive review: 0.0
Probability in negarive review: 0.0

disheartening
Probability in positive review: 3.261e-06
Probability in negarive review: 3.6518e-06



In [11]:
# Example 6 candidate words
candidates = ['lost', 'unscary', 'stylelessly', 'disheartening']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

awfully
Probability in positive review: 1.46743e-05
Probability in negarive review: 3.10403e-05

slow
Probability in positive review: 0.0001271775
Probability in negarive review: 0.0001259869

pestering
Probability in positive review: 1.6305e-06
Probability in negarive review: 1.8259e-06

loser
Probability in positive review: 2.11963e-05
Probability in negarive review: 4.19956e-05



In [14]:
# Example 7 candidate words
candidates = ['talent', 'notable', 'freedom', 'attracts']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

talent
Probability in positive review: 0.0001271775
Probability in negarive review: 0.0002191076

notable
Probability in positive review: 3.75011e-05
Probability in negarive review: 3.10403e-05

freedom
Probability in positive review: 7.66326e-05
Probability in negarive review: 1.8259e-05

attracts
Probability in positive review: 8.1524e-06
Probability in negarive review: 5.4777e-06



In [38]:
# Example 8 candidate words
candidates = ['abruptly', 'worst', 'pointless', 'wreck']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

abruptly
Probability in positive review: 6.5219e-06
Probability in negative review: 1.46072e-05

worst
Probability in positive review: 7.17412e-05
Probability in negative review: 0.0004181304

pointless
Probability in positive review: 2.60877e-05
Probability in negative review: 0.0001259869

wreck
Probability in positive review: 2.28267e-05
Probability in negative review: 2.73885e-05



In [43]:
# Example 9 candidates
candidates = ['watchable', 'holes', 'extraordinary', 'miracle']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

watchable
Probability in positive review: 9.7829e-06
Probability in negative review: 3.65179e-05

holes
Probability in positive review: 3.26096e-05
Probability in negative review: 5.84287e-05

extraordinary
Probability in positive review: 5.70668e-05
Probability in negative review: 1.64331e-05

miracle
Probability in positive review: 1.14134e-05
Probability in negative review: 5.4777e-06



In [50]:
# Example 10 candidates
candidates = ['tarantino', 'laugh', 'funny', 'great']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

tarantino
Probability in positive review: 0.0001190251
Probability in negative review: 2.00849e-05

laugh
Probability in positive review: 0.0001483738
Probability in negative review: 0.0002118041

funny
Probability in positive review: 0.0005592549
Probability in negative review: 0.0007212293

great
Probability in positive review: 0.0010956831
Probability in negative review: 0.0006244568

